In [1]:
import igraph 
import csv
import unicodedata

In [1]:
def CONGAYearAlgorithm(graph,result, Overlap):
    startedGraph = graph.copy()
    while(graph.ecount()>0):
        edge_betweenness = graph.edge_betweenness(directed=False, cutoff=None, weights=None)
        max_eb = max(edge_betweenness)
        #print(max_eb)
        vertex_betweenness = graph.betweenness(vertices=None, directed=False, cutoff=None, weights=None, nobigint=True)
        max_vb = max(vertex_betweenness)
        #print(max_vb)
        
        if(max_eb>=max_vb):
            edgesToDelete = [graph.es[idx].tuple for idx, eb in enumerate(edge_betweenness) if (eb == max_eb and eb == max_eb)]
            #print(edgesToDelete)
            result.append(edgesToDelete)
            #print(result)
            graph.delete_edges(edgesToDelete)
        else:
            vertexToCopy = vertex_betweenness.index(max_vb);
            #print(vertexToCopy)
            incident = graph.incident(vertexToCopy, mode=ALL)
            #print(incident)
            graph.add_vertices(1)
            graph.vs[graph.vcount()-1]['name'] = graph.vs[vertexToCopy]['name']
            Overlap[graph.vcount()-1] = vertexToCopy
            #print(Overlap)
            
            sum = 0
            for edge in incident:
                #print(graph.es[edge]['weight'])
                sum += graph.es[edge]['weight']

            average = sum/len(incident)
            #print(sum)
            #print(average)
            edges_to_delete = []
            new_vertex_counter = graph.vcount()-1
            allSameWeight = 1
            popWeight = -1
            for edge in incident:
                weight = graph.es[edge]['weight']
                if popWeight != weight and edge != incident[0]:
                    allSameWeight = 0
                if(weight<average):
                    edges_to_delete.append(edge)
                    source = graph.es[edge].source
                    target = graph.es[edge].target
                    #print([source, target])
                    if(target == vertexToCopy):
                        #print([(source, new_vertex_counter)])
                        graph.add_edges([(source, new_vertex_counter)])
                        graph.es[graph.ecount()-1]["weight"]=weight
                    else:
                        #print([(new_vertex_counter, target)])
                        graph.add_edges([(new_vertex_counter, target)])
                        graph.es[graph.ecount()-1]["weight"]=weight
                popWeight = weight
            if allSameWeight == 1:
                #print('All same weight!')
                edges_to_delete.append(edge)
                if vertexToCopy == target:
                    graph.add_edges([(graph.es[edge].source, new_vertex_counter)])
                else:
                    graph.add_edges([(new_vertex_counter,graph.es[edge].target)])
                graph.es[graph.ecount()-1]["weight"]=weight
                        
            toAdd = [(vertexToCopy, vertexToCopy)]
            for edge in edges_to_delete:
                toAdd.append((graph.es[edge].source,graph.es[edge].target))
            result.append(toAdd)
            graph.delete_edges(edges_to_delete)
        
        print(countModularity(startedGraph,graph,Overlap))
            #incident2 = graph.incident(vertexToCopy, mode=ALL)
            #print(incident2)
            #print(result)

In [2]:
def countModularity(startedGraph,graph, Overlap):
    modularityGraph = startedGraph.copy()
    while modularityGraph.ecount() > 0:
        for edge in modularityGraph.es:
            modularityGraph.delete_edges(edge)
        
    edges_to_add = []
    for edge in graph.es:
        source = edge.source
        target = edge.target
        if(source == target):
            continue
        if(source>=modularityGraph.vcount()):
            while(Overlap[source]!=-1):
                source=Overlap[source]
        if(target>=modularityGraph.vcount()):
            while(Overlap[target]!=-1):
                target=Overlap[target]
        edges_to_add.append((source,target))

    modularityGraph.add_edges(edges_to_add)
    
    m = 0
    for component in graph.components():
        community = []
        for v in component:
            #print(v)
            if(v>=modularityGraph.vcount()):
                while(Overlap[v]!=-1):
                    v=Overlap[v]
            community.append(v)
            #print(community)
        #print(community)
        m += modularity(modularityGraph, community)
        #print(m)

    return m/len(graph.components())

In [ ]:
def modularity(graph, community):
    M_in = 0
    M_out = 0
    for node in community:
        for edge_ind in graph.incident(node):
            edge = graph.es[edge_ind]
            #print(edge.source, edge.target, node)
            if node == edge.target and node == edge.source:
                continue
                #print("Warn - out vertex equals current vertex")
            elif node == edge.target:
                if edge.source in community:
                    M_in+=1
                else:
                    M_out+=1
            elif node  == edge.source:
                if edge.target in community:
                    M_in+=1
                else:
                    M_out+=1
            #print(M_in, M_out)
            #print(" ")
    if M_out == 0:
        #print("klika")
        return len(graph.vs)
    return M_in / 2 / M_out